# GCP Flight Function

---
## Install functions-framework 💾
If you haven't already installed functions-framework, you will need to for testing the cloud function locally. Uncomment the code below, install, and then recomment the code - you only need to install it once.

In [40]:
# !pip install functions-framework

## Definition of Flight Function

In [28]:
import functions_framework
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
import requests
from dotenv import load_dotenv
import os

@functions_framework.http
def GetFlightInfo(request):
    
   # Connection setup
    load_dotenv("variables.env")
    schema = os.getenv("schema")
    host = os.getenv("host")
    user = os.getenv("user")
    password = os.getenv("password")
    port = os.getenv("port")
    
    # Create connection string
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    # get icaos from database
    icao_df = pd.read_sql("airport", con=connection_string)
    icaos = list(icao_df['icao'])

    # gather flight arrival information with RapidAPI    
    RapidAPI_key = os.getenv("RapidAPI_key")
    
    origin_cities = []
    expected_arrival_times = []
    flight_status = []
    icao_list = []
    flight_num = []
    flight_info_df = pd.DataFrame({"origin":[] , "expected_arrival_time":[], "flight_status":[], "icao":[], "flight_number":[]})
    tomorrow = datetime.now() + timedelta(days=1)
    tomorrow_url_format = tomorrow.strftime('%Y-%m-%d')
    times_list = [["00:00", "11:59"],["12:00", "23:59"]]

    for icao in icaos:
        
        for times in times_list:
            
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow_url_format}T{times[0]}/{tomorrow_url_format}T{times[1]}"
            
            querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
            
            headers = {"X-RapidAPI-Key": RapidAPI_key, "x-rapidapi-host": "aerodatabox.p.rapidapi.com"}
            
            response = requests.get(url, headers=headers, params=querystring)
            
            for flight in response.json()['arrivals']:
                origin_cities.append(flight['departure']['airport']['name'])
                # Get arrival time (revised if available, otherwise scheduled)
                arrival = flight.get('arrival', {})
                revised_time = arrival.get('revisedTime', {})
                scheduled_time = arrival.get('scheduledTime', {})
            
                # Either get revised['local'] or scheduled['local']
                arrival_time = revised_time.get('local') or scheduled_time.get('local') or 'Unknown'
                expected_arrival_times.append(arrival_time)
                flight_status.append(flight['status'])
                icao_list.append(icao)
                flight_num.append(flight['number'])
    
    flight_info_df["origin"] = origin_cities
    flight_info_df["expected_arrival_time"] = expected_arrival_times
    flight_info_df["flight_status"] = flight_status
    flight_info_df["icao"] = icao_list
    flight_info_df["flight_number"] = flight_num

    SQL_Flight_df = pd.merge(flight_info_df, icao_df, on="icao")
    SQL_Flight_df['expected_arrival_time'] = pd.to_datetime(SQL_Flight_df['expected_arrival_time'])
    SQL_Flight_df.drop_duplicates(inplace=True)
    SQL_Flight_df.reset_index(drop=True, inplace=True)

    # upload final df with flight info to mySQL
    
    SQL_Flight_df.to_sql('flight',
                      if_exists='append',
                      con=connection_string,
                      index=False)
    

    return "Data has been sent to the table"

## Test Flight Function

In [30]:
from flask import Request
import json

request_data = {}
request = Request.from_values(data=json.dumps(request_data))

# Call the function
response = GetFlightInfo(request)
print(response)

Data has been sent to the table
